# Overview of this notebook
Takes fixations and drops post-choice fixations

In [1]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from scipy.stats import entropy

In [2]:
## Grab RTs for each Trial, Sub ##

# Load behavioral data
behav_path = os.getcwd() + '/ProcessedData/AllData.csv' # this expects ProcessedData to be in the analysis repo
behav_data = pd.read_csv(behav_path)
subjects = behav_data['Subj'].unique()

## Get metadata.
n_trials = max(behav_data['Trial'])
n_games = max(behav_data['Game'])
game_length = len(behav_data.loc[(behav_data['Subj'] == 23) & (behav_data['Game'] == 1)])

# change subj to int
behav_data['Subj'] = behav_data['Subj'].astype(int)

## Add trial-within-game variable.
behav_data['WithinGameTrial'] = behav_data['Trial'] - (behav_data['Game']-1)*game_length

rt_df = behav_data[['Subj', 'Trial','RT']]
rt_df

,Subj,Trial,RT
0,23,1,1.12700
1,23,2,1.08580
2,23,3,1.24420
3,23,4,0.47509
4,23,5,0.88090
...,...,...,...
15195,71,396,0.61252
15196,71,397,0.89898
15197,71,398,0.59331
15198,71,399,0.52019


In [3]:
## Define global variables ##
et_data_dir = os.getcwd() + '/ProcessedData/'# this expects ProcessedData to be in the analysis repo
image_dir = os.getcwd().strip('ldm-analysis') + 'FinalStimuli/ByNumber/'
raw_data_dir = os.getcwd().strip('ldm-analysis') + 'RawData/' # this expects RawData to be one directory up from the analysis repo

## Define screen metadata.
xdim, ydim, n_screens = 1280, 1024, 1 
aoisidelength = 162
n_aois = 9

# Define list with all subject ids; remove 33, 66, and 58 for ec and preproc troubles
good_subjects =  [23, 26, 27, 28, 29, 30, 31, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 49, 53, 54, 
                 55, 57, 60, 61, 62, 63, 67, 68, 69, 70, 71];

In [12]:
np.arange(400)+1

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
       105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
       118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130,
       131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143,
       144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156,
       157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169,
       170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 18

In [28]:
for subj_id in good_subjects:
    clean_fixation_df = pd.DataFrame()
    sub_rts = rt_df.loc[rt_df.Subj == subj_id]
    fixations = pd.read_csv(et_data_dir + str(subj_id) + 'fixations.csv')
    fixations['Trial'] = fixations.Trial.astype(int)
    for trial in np.arange(400)+1:
        trial_fixation_df = fixations.loc[fixations['Trial'] == trial]
        trial_rt = sub_rts.loc[sub_rts.Trial == trial].RT.item()
        clean_trial_fixation_df = trial_fixation_df.loc[trial_fixation_df['Onset'] < trial_rt]
        clean_fixation_df = clean_fixation_df.append(clean_trial_fixation_df, ignore_index=True)
    clean_fixation_df.to_csv(et_data_dir + str(subj_id) + 'pre-choice_fixations.csv', index=False)


In [4]:
for subj_id in good_subjects:
    clean_fixation_df = pd.DataFrame()
    sub_rts = rt_df.loc[rt_df.Subj == subj_id]
    fixations = pd.read_csv(et_data_dir + str(subj_id) + 'fixations.csv')
    fixations['Trial'] = fixations.Trial.astype(int)
    for trial in np.arange(400)+1:
        trial_fixation_df = fixations.loc[fixations['Trial'] == trial]
        trial_rt = sub_rts.loc[sub_rts.Trial == trial].RT.item()
        clean_trial_fixation_df = trial_fixation_df.loc[trial_fixation_df['Onset'] >= trial_rt]
        clean_fixation_df = clean_fixation_df.append(clean_trial_fixation_df, ignore_index=True)
    clean_fixation_df.to_csv(et_data_dir + str(subj_id) + 'post-choice_fixations.csv', index=False)

In [4]:
for subj_id in good_subjects:
    clean_fixation_df = pd.DataFrame()
    sub_rts = rt_df.loc[rt_df.Subj == subj_id]
    fixations = pd.read_csv(et_data_dir + str(subj_id) + 'fixations.csv')
    fixations['Trial'] = fixations.Trial.astype(int)
    for trial in np.arange(400)+1:
        trial_fixation_df = fixations.loc[fixations['Trial'] == trial]
        trial_rt = sub_rts.loc[sub_rts.Trial == trial].RT.item()
        outcome_start = max([trial_rt, 2])
        clean_trial_fixation_df = trial_fixation_df.loc[trial_fixation_df['Onset'] >= outcome_start]
        clean_fixation_df = clean_fixation_df.append(clean_trial_fixation_df, ignore_index=True)
    clean_fixation_df.to_csv(et_data_dir + str(subj_id) + 'outcome_fixations.csv', index=False)

In [31]:
fixations

,Unnamed: 0,Trial,AoI,Onset,Offset,Duration,Feature,Block
0,0,1,6.0,0.000,0.004,0.004,1.0,1
1,2,1,10.0,0.156,0.180,0.024,10.0,1
2,3,1,6.0,0.184,0.212,0.028,1.0,1
3,5,1,5.0,0.220,0.376,0.156,9.0,1
4,6,1,10.0,0.380,0.384,0.004,10.0,1
...,...,...,...,...,...,...,...,...
7552,1022,400,6.0,3.192,3.196,0.004,3.0,10
7553,1023,400,10.0,3.200,3.208,0.008,10.0,10
7554,1024,400,10.0,3.260,3.300,0.040,10.0,10
7555,1025,400,6.0,3.304,3.316,0.012,3.0,10


In [5]:
clean_fixation_df

,Unnamed: 0,Trial,AoI,Onset,Offset,Duration,Feature,Block
0,12,1,3.0,2.152,2.156,0.004,3.0,1
1,13,1,2.0,2.160,2.464,0.304,7.0,1
2,14,1,3.0,2.468,2.612,0.144,3.0,1
3,16,1,10.0,2.996,3.008,0.012,10.0,1
4,17,1,3.0,3.012,3.028,0.016,3.0,1
...,...,...,...,...,...,...,...,...
3607,1022,400,6.0,3.192,3.196,0.004,3.0,10
3608,1023,400,10.0,3.200,3.208,0.008,10.0,10
3609,1024,400,10.0,3.260,3.300,0.040,10.0,10
3610,1025,400,6.0,3.304,3.316,0.012,3.0,10
